# IMO 2025 Problem 6

**Prompt used**

Act as an expert in combinatorial optimization and computational geometry. Your task is to solve the following grid tiling problem.

Problem Statement:
Consider an n by n grid of unit squares. You need to place non-overlapping rectangular tiles on this grid. The sides of each tile must align with the grid lines. Your goal is to find the minimum number of tiles required to cover the grid such that every row and every column has exactly one unit square that is not covered by a tile.

Your Task:
You must write a Python search function that, for a given grid size n, finds the best possible arrangement of tiles. "Best" means it satisfies the condition and uses the fewest tiles. Your function should return a list of tiles, where each tile is represented by a tuple of four integers: (row, column, height, width). The input to your function will be an 'n' value, which is the grid size. This is guaranteed to be an integer.

Evaluation:
Your proposed tiling will be evaluated by the calculate_score function provided below. Your goal is to find a tiling that maximizes this score. A higher score means a better solution. The score is designed to heavily penalize solutions that violate the problem's constraints (e.g., overlapping tiles, tiles out of bounds, or incorrect number of uncovered squares per row/column). For a valid solution, the score is simply the negative of the number of tiles used, so maximizing the score is equivalent to minimizing the number of tiles.

Here is the scoring function you will be evaluated against:

```
def calculate_score(tiles: List[Tuple[int, int, int, int]], n: int) -> float:
    """
    Calculates the score for a given tiling configuration.
    A higher score is better.
    """
    # Basic type and format checks
    [...]

    # Core logic (JIT-compiled for speed)
    grid = np.zeros((n, n), dtype=np.int32)
    [Fill up the grid with the tiles]

    uncovered_per_row = n - np.sum(grid, axis=1)
    uncovered_per_col = n - np.sum(grid, axis=0)

    row_penalty = np.sum(np.abs(uncovered_per_row - 1))
    col_penalty = np.sum(np.abs(uncovered_per_col - 1))
    total_penalty = row_penalty + col_penalty

    score = -tiles_array.shape[0] - 1000 * total_penalty
    return float(score)
```


You can call this calculate_score function as many times as you want within your search function. You don't have to implement this, it's already found elsewhere in the codebase.

Search Function Requirements:
Your search function will be given 10 seconds to run. After this time, it must return the best list of tiles it has found. Structure your search with a main loop like while time.time() - start_time < 10: to manage the runtime.

Big important hint: you must try to find a general solution to the problem. Instead of focusing on the search, I would strongly encourage you to try to find a general solution to the problem. Your program will be evaluated on some very large values of n, where the search won't help you -- try to find the pattern instead that works for all n.

Second hint: much better configurations are possible than the program you are shown. The patterns you have to discover are not hard, you don't need to use any deep results at all. Always try to find a better pattern, don't be scared of the difficult sounding problem, once you see the solution you'll realise it wasn't hard at all. Good luck, I believe in you, but you also have to believe in yourself!

Good luck!

In [ ]:
#@title Verification function used

import itertools
import logging
import time
from scipy import integrate
import numpy as np
from scipy import optimize
import warnings
import random
import re
from collections.abc import Callable, Mapping
from typing import Any, List, Tuple
import scipy.linalg as la
import collections
import copy
import math
import numba
import numpy as np
from scipy.optimize import milp, LinearConstraint, Bounds
from itertools import product

njit = numba.njit

# Gemini written scoring function
@njit
def _calculate_score_numba(tiles_array: np.ndarray, n: int) -> float:
  """Core scoring logic, JIT-compiled with Numba for performance."""
  grid = np.zeros((n, n), dtype=np.int32)

  # Check for out-of-bounds tiles and overlaps.
  for i in range(tiles_array.shape[0]):
    r, c, h, w = tiles_array[i, :]
    if r < 0 or c < 0 or r + h > n or c + w > n:
      return -np.inf  # Out of bounds.
    if np.any(grid[r : r + h, c : c + w]):
      return -1_000_000_000.0  # Overlap.
    grid[r : r + h, c : c + w] = 1

  # Calculate penalty for deviating from the 'one-uncovered-square' rule.
  uncovered_per_row = n - np.sum(grid, axis=1)
  uncovered_per_col = n - np.sum(grid, axis=0)

  row_penalty = np.sum(np.abs(uncovered_per_row - 1))
  col_penalty = np.sum(np.abs(uncovered_per_col - 1))
  total_penalty = row_penalty + col_penalty

  # The score is primarily driven by the penalty. For valid solutions (penalty=0),
  # the score is the negative number of tiles, encouraging fewer tiles.
  score = -tiles_array.shape[0] - 100 * total_penalty
  return float(score)


def calculate_score(tiles: List[Tuple[int, int, int, int]], n: int) -> float:
  """Calculates the score for a given tiling configuration.

  A higher score is better.
  """
  # Perform initial validation in Python before passing to Numba.
  if not isinstance(tiles, list):
    return -np.inf
  if not tiles:
    # Calculate penalty for an empty grid.
    penalty = 2 * n * (n - 1)
    return float(-1000 * penalty)

  for t in tiles:
    if not isinstance(t, (list, tuple)) or len(t) != 4:
      return -np.inf
    if not all(isinstance(x, int) for x in t):
      return -np.inf
    if t[2] <= 0 or t[3] <= 0:
      return -np.inf  # Non-positive dimensions.

  tiles_array = np.array(tiles, dtype=np.int32)
  return _calculate_score_numba(tiles_array, n)


In [ ]:
#@title AlphaEvolve's solution (one example)

# This is an example where AlphaEvolve found the correct solution without
# really understanding what it's doing

"""AlphaEvolve experiment for the grid tiling problem."""

def search_for_best_tiling(
    n: int,
    max_runtime: int = 10,
) -> List[Tuple[int, int, int, int]]:
    """Tries several promising constructive heuristics and returns the best tiling found.

    This approach abandons local search in favor of testing a curated set of
    global constructions, which is more effective for this problem. The candidates are:
    1. Explicitly added strong general patterns (odd/even n specific permutations).
    2. The p(i) = k*i mod n family of permutations for small k coprime to n,
       including diagonal (k=1) and shifted versions.
    3. Divisor-based interleaved permutations, which generalize special cases for even n.
    4. Anti-diagonal permutation p(i) = n-1-i.
    5. Permutations based on k near sqrt(n).

    For each candidate permutation of uncovered squares, a greedy tiling algorithm
    is used to cover the rest of the grid. The best result across all candidates
    is returned. This is a deterministic process that efficiently explores
    structurally different and promising solutions.
    """
    if n == 1:
        return []

    @njit
    def _find_and_mark_tiles(
        n: int, uncovered_cols: List[int]
    ) -> List[Tuple[int, int, int, int]]:
        """
        Greedily tiles the grid, avoiding the squares specified by `uncovered_cols`.
        """
        covered_grid = np.zeros((n, n), dtype=np.bool_)
        for r in range(n):
            c = uncovered_cols[r]
            covered_grid[r, c] = True

        tiles = []
        for r in range(n):
            for c in range(n):
                if not covered_grid[r, c]:
                    max_w = 0
                    while c + max_w < n and not covered_grid[r, c + max_w]:
                        max_w += 1

                    max_h = 1
                    while r + max_h < n:
                        is_row_clear = True
                        for w_offset in range(max_w):
                            if covered_grid[r + max_h, c + w_offset]:
                                is_row_clear = False
                                break
                        if not is_row_clear:
                            break
                        max_h += 1

                    tile = (r, c, max_h, max_w)
                    tiles.append(tile)

                    for row_offset in range(max_h):
                        for col_offset in range(max_w):
                            covered_grid[r + row_offset, c + col_offset] = True
        return tiles

    candidate_perms = []

    # Candidate 1: p(i) = k*i mod n. We test small k coprime to n.
    # We limit the number of k's to test to keep the search fast.
    # Candidate 1: p(i) = k*i mod n and p(i) = (k*i + n//2) mod n.
    # We test more k values and also include shifted versions.
    num_k_to_test = min(n, 20) # Test up to 20 coprime k values, or n-1 if n is small
    tested_k_count = 0
    for k in range(1, n):
        if tested_k_count >= num_k_to_test:
            break
        if math.gcd(k, n) == 1:
            # Original k*i mod n permutation
            perm_k_original = [(k * i) % n for i in range(n)]
            if perm_k_original not in candidate_perms:
                candidate_perms.append(perm_k_original)

            # Shifted k*i mod n permutation
            shift_offset = n // 2
            if shift_offset > 0: # Only add shifted if offset is meaningful
                perm_k_shifted = [(k * i + shift_offset) % n for i in range(n)]
                if perm_k_shifted not in candidate_perms:
                    candidate_perms.append(perm_k_shifted)
            tested_k_count += 1

    # Candidate 2: Divisor-based interleaved permutations.
    # This generalizes the strong permutation for even n to any n with divisors.
    divisors = set()
    for i in range(2, int(math.sqrt(n)) + 1):
        if n % i == 0:
            divisors.add(i)
            if i * i != n:
                divisors.add(n // i)

    for m in sorted(list(divisors)):
        if m >= n: continue
        k = n // m
        # Construction p(m*i+j) = i + pi(j)*k. Test two permutations for pi.
        # pi = identity
        perm_id = [0] * n
        for i in range(k):
            for j in range(m):
                perm_id[m * i + j] = i + j * k
        if perm_id not in candidate_perms:
            candidate_perms.append(perm_id)

        # pi = reverse (this generalizes the old special case for even n)
        perm_rev = [0] * n
        for i in range(k):
            for j in range(m):
                perm_rev[m * i + j] = i + (m - 1 - j) * k
        if perm_rev not in candidate_perms:
            candidate_perms.append(perm_rev)

    # Candidate 3: Anti-diagonal permutation.
    candidate_perms.append([n - 1 - i for i in range(n)])

    # Candidate 4: Permutations based on k ~ sqrt(n)
    # This tries to find a 'spread' that scales non-linearly with n.
    k_approx_sqrt = round(math.sqrt(n))
    for k_val in [k_approx_sqrt - 1, k_approx_sqrt, k_approx_sqrt + 1]:
        if k_val > 0 and math.gcd(k_val, n) == 1:
            perm = [(k_val * i) % n for i in range(n)]
            if perm not in candidate_perms:
                candidate_perms.append(perm)

    # Removed candidate for "4n/3" construction as the implemented permutation
    # was found to be suboptimal for n=3 (6 tiles vs. 4 for diagonal) and n=6 (12 tiles).
    # The diagonal (k=1) permutation already covers the optimal n=3 case.

    best_tiles = []
    # Initialize with a very bad score.
    best_score = -float('inf')

    for perm in candidate_perms:
        tiles = _find_and_mark_tiles(n, perm)
        score = calculate_score(tiles, n)
        if score > best_score:
            best_score = score
            best_tiles = tiles

    return best_tiles


In [ ]:
#@title Visualizing AlphaEvolve's solution

# Gemini written plotting function
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from typing import List, Tuple

def plot_tiling(tiling: List[Tuple[int, int, int, int]], n: int, figsize: Tuple[int, int] = (8, 8)):
    """
    Plots the tiling solution on an n x n grid with random colors for each tile.

    Args:
        tiling: A list of tiles, where each is a tuple (row, col, height, width).
        n: The size of the grid.
        figsize: The size of the matplotlib figure.
    """
    fig, ax = plt.subplots(1, figsize=figsize)

    # --- Set up the grid ---
    ax.set_xlim(0, n)
    ax.set_ylim(0, n)
    ax.set_xticks(np.arange(n + 1))
    ax.set_yticks(np.arange(n + 1))
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.grid(True, which='both', color='k', linestyle='-', linewidth=1)
    ax.invert_yaxis()
    ax.set_aspect('equal', adjustable='box')

    # --- Generate a list of random colors, one for each tile ---
    # This is the main change from the previous version.
    # np.random.rand(3) creates a random RGB color.
    random_colors = [np.random.rand(3) for _ in range(len(tiling))]

    # --- Plot the tiles ---
    for i, (r, c, h, w) in enumerate(tiling):
        rect = patches.Rectangle(
            (c, r), w, h,
            facecolor=random_colors[i],  # Use the pre-generated random color
            edgecolor='black',
            linewidth=1.5,
            alpha=0.8
        )
        ax.add_patch(rect)

    # --- Identify and mark uncovered squares ---
    grid_coverage = np.zeros((n, n), dtype=int)
    for (r, c, h, w) in tiling:
        grid_coverage[r:r+h, c:c+w] = 1

    uncovered_count = 0
    for r in range(n):
        for c in range(n):
            if grid_coverage[r, c] == 0:
                uncovered_count += 1
                ax.text(c + 0.5, r + 0.5, 'X',
                        ha='center', va='center',
                        fontsize=16, color='red', weight='bold')

    # --- Final Touches ---
    ax.set_title(
        f"Tiling for n={n}\n{len(tiling)} tiles. Optimal: {n + 2*math.sqrt(n)-3}",
        fontsize=16
    )
    plt.tight_layout()
    plt.show()


def evaluate():
  """Evaluates a tiling for the given grid size `n`."""

  ns_to_test = list(range(2,11)) + [45]
  our_scores = []
  optimal_scores = []
  for n in ns_to_test:
    best_tiling = search_for_best_tiling(n**2)
    plot_tiling(best_tiling, n**2)


evaluate()